En este notebook se hara el proceso de **PRIORIZAR** , se tomaran las PQR Desde diciembre del 2022 a  **2023 a Marzo(periodo de tres meses)**


# CONFIGURAR SPARK

In [ ]:
import pandas as pd
import sys
sys.path.append('../librerias')


# LIBRERIAS





In [ ]:
!python -m spacy download es_core_news_sm

!python -m spacy download es_core_news_lg

In [2]:
import pandas as pd

from google.colab import drive
import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification




nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing


#.master("local[2]")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

# ETL

In [13]:

pandas_c_F= pd.read_csv('/content/PRIORIZAR/entranamiento_/binario_entrenamiento_cruda.csv',delimiter=';')

pandas_c_F= pandas_c_F.fillna("vacios")


pandas_c_F

,index,FECHA_CREACION,NUMERO_PQR,DESCRIPCION_PQR,clasificacion
0,0,1/12/22 5:57,MDM-PQR-35424576,Linea: 6013842997 Nombre: LINA MARIA CESPEDES ...,1.0
1,1,1/12/22 7:06,MDM-PQR-35425135,Gestión de llamada:Se valida en factura que es...,1.0
2,2,1/12/22 7:28,MDM-PQR-35425426,Nombre de quien se comunica:WALTER ARROYO CAIC...,1.0
3,3,1/12/22 7:46,MDM-PQR-35425697,APP MIETB// INFORMACIÓN DE PAGO DOBLE SE L...,1.0
4,4,1/12/22 7:56,MDM-PQR-35425883,ASEG// LT 6013350275 //EXPLICACIÓN CONCEPTOS F...,1.0
...,...,...,...,...,...
15520,15520,17/02/23 11:36,MDM-PQR-37150712,linea 6017628666 cliente URIEL ERNESTO GERENA ...,0.0
15521,15521,17/02/23 11:39,MDM-PQR-37150816,nombre: MARISOL ANGELICA CRUZ RODRIGUEZ cc: ...,0.0
15522,15522,17/02/23 11:51,MDM-PQR-37151180,nombre: ANA GLADYS PARADA VARGAS cc: 35464758 ...,0.0
15523,15523,17/02/23 11:51,MDM-PQR-37151190,ct solicita envio de factura se envia documento,0.0


In [ ]:
pandas_c_F.isnull().sum()

In [ ]:
from clean_text import clean_text

In [ ]:
#declare answers and questions
descripcion=pandas_c_F.DESCRIPCION_PQR	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#restar dos para optener la data de excel y poder comparar

clean_descripcion  [1002]

In [ ]:
len(clean_descripcion)

In [ ]:
from tqdm import tqdm
from spelling import correct_sentence

In [ ]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

In [ ]:
corrected_sentences1=corrected_sentences

In [4]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(corrected_sentences1)


clean_descripcion[883]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


11659


['reclamar', 'factura']

In [7]:
clean_descripcion[100]

['informar',
 'siembre',
 'pago',
 'factura',
 'desear',
 'validar',
 'llego',
 'ahora',
 'pagar']

In [ ]:
import pickle



# Specify the file path where you want to save the list
file_path = "/content/PRIORIZAR/data/entrenamiento_binario.pkl"

# Open the file in binary write mode
with open(file_path, "wb") as file:
    # Use the pickle.dump() function to save the list to the file
    pickle.dump(clean_descripcion, file)

In [ ]:
import pickle

# Specify the file path from where you want to load the list
file_path = "/content/PRIORIZAR/entranamiento_/entrenamiento_binario_final.pkl"

# Open the file in binary read mode
with open(file_path, "rb") as file:
    # Use the pickle.load() function to load the list from the file
    loaded_list = pickle.load(file)

print(loaded_list)

In [ ]:

clean_descripcion=loaded_list

In [8]:
len(clean_descripcion)

15525

In [11]:
clean_descripcion[5675]

['llamar', 'promocion', 'plan', 'factura']

creacion tabla

In [ ]:
import numpy as np
import pandas as pd
from unidecode import unidecode



merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion']).reset_index()

#############################################

pandas_c_F.reset_index(inplace=True)

#merged_df = pd.merge(merged_df, pandas_c_F, on='index', how='inner').loc[:, ['FECHA_CREACION','NUMERO_PQR','descripcion', 'clasificacion']]

merged_df = pd.concat([merged_df, pandas_c_F], axis=1)



shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)




#shuffled_df.rename(columns={"FECHA_CREACION": "event_timestamp"}, inplace=True)

#string_values = ['abcde', 'fghij', 'klmno', 'pqrst', 'uvwxy', 'zabcd', 'efghi', 'jklmn', 'opqrs', 'tuvwx']



# Función para eliminar las tildes de un texto
def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
shuffled_df['descripcion'] = shuffled_df['descripcion'].apply(remove_accents)



# Randomly select string values to fill the DataFrame
#random_strings = np.random.choice(string_values, size=len(shuffled_df))

#shuffled_df["NUMERO_PQR"]=random_strings


shuffled_df

In [33]:
shuffled_df.to_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv', index=False)

shuffled_df= shuffled_df.fillna(0)

shuffled_df

,index,descripcion,level_0,index,FECHA_CREACION,NUMERO_PQR,DESCRIPCION_PQR,clasificacion
0,777,concepto facturar,777,777,12/12/22 9:54,MDM-PQR-35630165,NOMBRE: HILDEBRANDO MARTINEZ ROJAS CC: 40554...,1.0
1,12104,facturacion,12104,12104,19/01/23 13:13,MDM-PQR-36420239,Línea:6012182752 Nombre de quien se comunica:...,0.0
2,148,sabria dannasanabriagmailcom solicitar factura...,148,148,2/12/22 11:07,MDM-PQR-35458749,DANNA STEFANIA SANABRIA RINCON 1000459282 31...,1.0
3,7187,suave facturacion acerca solicitar aumento fac...,7187,7187,16/02/23 9:23,MDM-PQR-37122403,6016452449 JOSE JOAQUÍN MARTÍNEZ SUÁREZ CC 2...,1.0
4,10515,generar aumento factura,10515,10515,30/12/22 16:58,MDM-PQR-35992356,Línea: SR-29338498 Nombre: JULIAN DAVID AREVA...,0.0
...,...,...,...,...,...,...,...,...
15520,10260,validar compensar falla masivo facturacion,10260,10260,27/12/22 22:27,MDM-PQR-35924209,se comunica para valiadr como es la compensaio...,0.0
15521,6484,solicitar validar concepto facturar,6484,6484,8/02/23 15:55,MDM-PQR-36957742,6013579780//SONIA EUGENIA CORTES//52158204// c...,1.0
15522,13598,manifestar retirar winsport quitar directv aun...,13598,13598,30/01/23 13:25,MDM-PQR-36695478,Línea: 6017913909 12054568653 Nombre de quien...,0.0
15523,1232,validacion factura nunca llegar acordado,1232,1232,19/12/22 11:16,MDM-PQR-35760641,Línea:6012562478 Nombre:LILIANA VILLAMIL JIME...,1.0


In [35]:
shuffled_df = pd.read_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv',delimiter=';')

In [37]:
from tfidf import TFIDF

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import dump



X = shuffled_df['descripcion']

y = shuffled_df['clasificacion']




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.9  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)



print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)



print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)


dump(value=pca, filename="/content/PRIORIZAR/modelo/pca_binario.joblib")

tf-idf with 3833 features
train with old features:  (12420, 3833)
train with new features: (12420, 784)
test with old features:  (3105, 3833)
test with new features: (3105, 784)
Training Data : (12420, 3833)
Testing Data :  (3105, 3833)


['/content/PRIORIZAR/modelo/pca_binario.joblib']

In [39]:
n_components

784

In [ ]:
pip install optuna

In [40]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score


In [41]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 60, 190),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }




    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy




In [42]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

[I 2023-07-06 20:32:07,010] A new study created in memory with name: lightgbm


[LightGBM] [Warning] bagging_fraction is set=0.4554031568612787, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4554031568612787
[LightGBM] [Warning] lambda_l2 is set=0.010007110473405195, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.010007110473405195
[LightGBM] [Warning] lambda_l1 is set=0.879294185249291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.879294185249291
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.48805353449026784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48805353449026784


[I 2023-07-06 20:32:17,116] Trial 0 finished with value: 0.7919484702093398 and parameters: {'n_estimators': 95, 'lambda_l1': 0.879294185249291, 'lambda_l2': 0.010007110473405195, 'num_leaves': 48, 'feature_fraction': 0.48805353449026784, 'bagging_fraction': 0.4554031568612787, 'bagging_freq': 2, 'learning_rate': 0.060733643622582864, 'min_child_samples': 43}. Best is trial 0 with value: 0.7919484702093398.


[LightGBM] [Warning] bagging_fraction is set=0.4164325559187557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4164325559187557
[LightGBM] [Warning] lambda_l2 is set=0.7069470359901728, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7069470359901728
[LightGBM] [Warning] lambda_l1 is set=0.1353297938831623, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1353297938831623
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9268704618345672, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9268704618345672


[I 2023-07-06 20:32:34,233] Trial 1 finished with value: 0.7916264090177134 and parameters: {'n_estimators': 117, 'lambda_l1': 0.1353297938831623, 'lambda_l2': 0.7069470359901728, 'num_leaves': 42, 'feature_fraction': 0.9268704618345672, 'bagging_fraction': 0.4164325559187557, 'bagging_freq': 5, 'learning_rate': 0.06503828814202763, 'min_child_samples': 58}. Best is trial 0 with value: 0.7919484702093398.


...

[I 2023-07-06 20:37:29,363] Trial 17 finished with value: 0.7938808373590982 and parameters: {'n_estimators': 168, 'lambda_l1': 4.208823363521199, 'lambda_l2': 2.7129182650340886, 'num_leaves': 88, 'feature_fraction': 0.5681857837622545, 'bagging_fraction': 0.5522268058119942, 'bagging_freq': 3, 'learning_rate': 0.04705291220874492, 'min_child_samples': 73}. Best is trial 12 with value: 0.8003220611916264.


[LightGBM] [Warning] bagging_fraction is set=0.6729756320321, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6729756320321
[LightGBM] [Warning] lambda_l2 is set=0.6320219749145862, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6320219749145862
[LightGBM] [Warning] lambda_l1 is set=0.3780711814912514, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.3780711814912514
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.6844806167350638, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6844806167350638


[I 2023-07-06 20:37:47,201] Trial 18 finished with value: 0.7903381642512077 and parameters: {'n_estimators': 154, 'lambda_l1': 0.3780711814912514, 'lambda_l2': 0.6320219749145862, 'num_leaves': 81, 'feature_fraction': 0.6844806167350638, 'bagging_fraction': 0.6729756320321, 'bagging_freq': 1, 'learning_rate': 0.05622573542141071, 'min_child_samples': 95}. Best is trial 12 with value: 0.8003220611916264.


[LightGBM] [Warning] bagging_fraction is set=0.5359007232429539, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5359007232429539
[LightGBM] [Warning] lambda_l2 is set=2.0862962817737, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.0862962817737
[LightGBM] [Warning] lambda_l1 is set=0.010398231057343471, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.010398231057343471
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.40607903153949193, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40607903153949193


[I 2023-07-06 20:37:59,218] Trial 19 finished with value: 0.794524959742351 and parameters: {'n_estimators': 175, 'lambda_l1': 0.010398231057343471, 'lambda_l2': 2.0862962817737, 'num_leaves': 54, 'feature_fraction': 0.40607903153949193, 'bagging_fraction': 0.5359007232429539, 'bagging_freq': 2, 'learning_rate': 0.04732125031248334, 'min_child_samples': 78}. Best is trial 12 with value: 0.8003220611916264.


In [43]:
print('Best parameters:', study.best_params)

Best parameters: {'n_estimators': 200, 'lambda_l1': 0.010538661525506412, 'lambda_l2': 4.2105515010394745, 'num_leaves': 31, 'feature_fraction': 0.7904508631838858, 'bagging_fraction': 0.7487452070745508, 'bagging_freq': 1, 'learning_rate': 0.04002333100026529, 'min_child_samples': 74}


In [44]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

[LightGBM] [Warning] lambda_l1 is set=0.010538661525506412, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.010538661525506412
[LightGBM] [Warning] feature_fraction is set=0.7904508631838858, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7904508631838858
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=4.2105515010394745, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.2105515010394745
[LightGBM] [Warning] bagging_fraction is set=0.7487452070745508, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7487452070745508


LGBMClassifier(bagging_fraction=0.7487452070745508, bagging_freq=1,
               feature_fraction=0.7904508631838858,
               lambda_l1=0.010538661525506412, lambda_l2=4.2105515010394745,
               learning_rate=0.04002333100026529, min_child_samples=74,
               n_estimators=200)

In [45]:
y_pred = mode.predict(X_test_new)

In [46]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.77      0.82      0.79      1447
         1.0       0.83      0.78      0.81      1658

    accuracy                           0.80      3105
   macro avg       0.80      0.80      0.80      3105
weighted avg       0.80      0.80      0.80      3105



In [47]:
from joblib import dump
dump(value=mode, filename="/content/PRIORIZAR/modelo/modelo_binario.joblib")

['/content/PRIORIZAR/modelo/modelo_binario.joblib']